# Units, scaling, non-dimensionalization

This notebook demonstrates how the dimensional parameater and non-dimensionalization is handled in UWsubduction. 


### Pint / Scaling

In the UWsubduction module, we make use of the underworld2 scaling module, which allows us to work with a dimensional quantities, and automates the non-dimensionlisation process.  



The underworld2 scaling module is a simple wrapper around the Pint library. More details can be found here:

https://github.com/hgrecco/pint

https://github.com/underworldcode/underworld2/blob/master/docs/examples/1_14_ScalingExample.ipynb

https://github.com/rbeucher/UWGeodynamics/tree/master/UWGeodynamics/scaling

### edicts

The UWsubduction module makes often makes use of a special form of python dictionary from the EasyDict module. These support an attribute-like notation (dot) for dictionary keys/values (https://github.com/makinacorpus/easydict)


### Import a set of dimensional parameters in an EasyDict

In [1]:
import numpy as np
import pint
from easydict import EasyDict as edict


import warnings; warnings.simplefilter('ignore')
from UWsubduction.params.minimal_example import pd, md, u, scaling

In [12]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import Terminal256Formatter
from pprint import pformat

def pprint_color(obj):
    print(highlight(pformat(obj), PythonLexer(), Terminal256Formatter()))

In [13]:
print(type(pd))

pprint_color(pd.refDensity)      #easydict style query 
pprint_color(pd['refDensity'])   #normal dictionary query 


<class 'easydict.EasyDict'>
<Quantity(3300.0, 'kilogram / meter ** 3')>

<Quantity(3300.0, 'kilogram / meter ** 3')>



Notice that the dictionary key `refDensity` has a corresponding value which is a Pint Quantity, i.e it has both magnitude and units. 

## Non-dimensionalisation

We now define a system of scaling factors, and use this to non-dimensionlize our paramter set. Let's set one of these systems up:

In [15]:
#import unsupported_dan.geodynamics.scaling as test_scaling;
#from underworld import scaling as sca
#sca.units

scaling_coefficients = scaling.get_coefficients()

In [18]:
print(pprint_color(scaling_coefficients))


#instead of importing from the params submodule, we'll explicity set the scaling values
KL = pd.refLength
KT = pd.potentialTemp - pd.surfaceTemp
Kt = KL**2/pd.refDiffusivity            #we use a diffusive time scale 
KM = pd.refViscosity * KL * Kt

scaling_coefficients["[length]"]      = KL.to_base_units()
scaling_coefficients["[temperature]"] = KT.to_base_units()
scaling_coefficients["[mass]"]        = KM.to_base_units()
scaling_coefficients["[time]"] =        Kt.to_base_units()

{'[length]': <Quantity(1.0, 'meter')>,
 '[mass]': <Quantity(1.0, 'kilogram')>,
 '[substance]': <Quantity(1.0, 'mole')>,
 '[temperature]': <Quantity(1.0, 'kelvin')>,
 '[time]': <Quantity(31556925.9747, 'second')>}

None


In [19]:
#print(pd.refLength)
print(scaling.non_dimensionalise(pd.refLength))

DimensionalityError: Cannot convert from '2900000.0 meter' ([length]) to 'a quantity of' ([length])

Because the default length scale is 1 meter, all that happenened is that the Pint Quantity was transformed into it's base (SI) units and divided by the scaling factor, leaving the dimensionless value (2900000.0)

### Scaling of mantle thermo-mechanical subduction

In order to non-dimensionalise the system we choose a system of scaling factors that are commonly used in thermal convection/ subduction studies. 

This system uses the convective layer depth (the mantle) as the length scale, a thermal diffusion time scale, and a stress (& mass) scale based on a viscous stress.

length Scale:
$L_s = L_r$

time scale: $t_s = \frac{L_s^2}{\kappa_r}$

stress scale $\sigma_s = \frac{\kappa_r \eta_r}{L_s}$

mass scale: $M_s = \frac{\eta_r L_s^3}{\kappa_r}$

Note that we only need the stress __or__ the mass scale to define a complete scaling


In [49]:
#import the default scaling
#from unsupported_dan.UWsubduction.default_scaling import sub_scaling


#instead of importing from the params submodule, we'll explicity set the scaling values
KL = pd.refLength
KT = pd.potentialTemp - pd.surfaceTemp
Kt = KL**2/pd.refDiffusivity            #we use a diffusive time scale 
KM = pd.refViscosity * KL * Kt

scaling_coefficients["[length]"]      = KL.to_base_units()
scaling_coefficients["[temperature]"] = KT.to_base_units()
scaling_coefficients["[mass]"]        = KM.to_base_units()
scaling_coefficients["[time]"] =        Kt.to_base_units()


In [25]:
sub_scaling.scaling['[length]']

<Quantity(2900000.0, 'meter')>

In [12]:
print(sub_scaling.nonDimensionalize(paramDict_dim.refLength))

1.0


We also provide a function to convert a dictionary of dimensional paramters to a dictionary of dimesnsionless parameters:

In [20]:
from unsupported_dan.UWsubduction.default_scaling import build_nondim_dict

paramDict_nondim = build_nondim_dict(paramDict_dim, sub_scaling)

In [21]:
print(paramDict_nondim.refLength)

1.0


### Temperature convention (offset)

One difficulty with allowing the scaling module to automate the non-dimensionalisation process is that we sometimes want to define an offset as well as a scaling. For intaance, it is common in thermal (Raylieigh-Bernard type) convection studies to deine the dimensionless temperature as:

$T' = \frac{T - T_0}{T_1 -T_0}$ 

Where $T_0$ is the surface temperature and T1, the base (or interior) temperature. 

At this stage, we have tried to adhere to this convention, by providing two sets of temperatures:


In [16]:
print(paramDict_dim.potentialTemp)
print(paramDict_dim.potentialTemp_)

1573.0 kelvin
1300.0 kelvin


In [17]:
print(sub_scaling.nonDimensionalize(paramDict_dim.potentialTemp))
print(sub_scaling.nonDimensionalize(paramDict_dim.potentialTemp_))

1.21
1.0


Where the `_` notation desribes the offset temperatures (dimensional and dimensionless)

We mention this to clarify (hopefully) the appearance of `potentialTemp_` and `surfaceTemp_` in the examples.

### Mininal example

 The subduction examples provided in this repository (/UWsubduction/Examples) are based on a set of paramters which we story in the script called `minimal_example.py`
 
In fact, we generally use two dictionaries, one storing physical consntants and paramters (for insnace flow law parameters,), the other storing paramters related more to modelling. Note that many of the modeling paramters have phsycial units, an example is the depth at to which the initial slab extends. 

In [39]:
#physical paramters
pprint_color(paramDict_dim)

{'adiabaticTempGrad': <Quantity(0.000369851247601, 'kelvin * kilogram * meter / joule / second ** 2')>,
 'cohesionMantle': <Quantity(20.0, 'megapascal')>,
 'diffusionEnergy': <Quantity(300000.0, 'joule / mole')>,
 'diffusionEnergyDepth': <Quantity(36083.714217, 'kelvin')>,
 'diffusionPreExp': <Quantity(5.34e-10, '1 / pascal / second')>,
 'diffusionVolume': <Quantity(5e-06, 'meter ** 3 / mole')>,
 'diffusionVolumeDepth': <Quantity(0.019449121963, 'kelvin / meter')>,
 'frictionMantle': <Quantity(0.2, 'dimensionless')>,
 'frictionMantleDepth': <Quantity(6468.0, 'kilogram / meter ** 2 / second ** 2')>,
 'gasConstant': <Quantity(8.314, 'joule / kelvin / mole')>,
 'lowerMantleViscFac': <Quantity(30.0, 'dimensionless')>,
 'potentialTemp': <Quantity(1573.0, 'kelvin')>,
 'potentialTemp_': <Quantity(1300.0, 'kelvin')>,
 'refDensity': <Quantity(3300.0, 'kilogram / meter ** 3')>,
 'refDiffusivity': <Quantity(1e-06, 'meter ** 2 / second')>,
 'refExpansivity': <Quantity(3e-05, '1 / kelvin')>,
 'refG

In [40]:
#modelling paramters
pprint_color(modelDict_dim)

{'aspectRatio': 5.0,
 'buoyancyFac': 1.0,
 'depth': <Quantity(1000, 'kilometer')>,
 'druckerAlpha': 1.0,
 'elementType': 'Q1/dQ0',
 'faultDestroyDepth': <Quantity(500, 'kilometer')>,
 'faultLocFac': 1.0,
 'faultThickness': <Quantity(10.0, 'kilometer')>,
 'faultViscDepthTaperStart': <Quantity(100, 'kilometer')>,
 'faultViscDepthTaperWidth': <Quantity(20, 'kilometer')>,
 'faultViscHorizTaperStart': <Quantity(300, 'kilometer')>,
 'faultViscHorizTaperWidth': <Quantity(300, 'kilometer')>,
 'lowerMantleDepth': <Quantity(660.0, 'kilometer')>,
 'lowerMantleTransWidth': <Quantity(10.0, 'kilometer')>,
 'meshRefineFactor': 0.7,
 'nltol': 0.01,
 'opAgeAtTrench': <Quantity(35.0, 'megayear')>,
 'penaltyMethod': True,
 'ppc': 25,
 'radiusOfCurv': <Quantity(350.0, 'kilometer')>,
 'refineHoriz': True,
 'refineVert': True,
 'res': 48,
 'slabAge': <Quantity(70.0, 'megayear')>,
 'slabInitMaxDepth': <Quantity(150, 'kilometer')>,
 'subZoneLoc': <Quantity(-100, 'kilometer')>,
 'viscosityMax': <Quantity(1e+25